# Forecasting Net Prophet

You’re a growth analyst at [MercadoLibre](http://investor.mercadolibre.com/about-us). With over 200 million users, MercadoLibre is the most popular e-commerce site in Latin America. You've been tasked with analyzing the company's financial and user data in clever ways to make the company grow. So, you want to find out if the ability to predict search traffic can translate into the ability to successfully trade the stock.

The instructions for this Challenge are divided into four steps, as follows:

* Step 1: Find unusual patterns in hourly Google search traffic

* Step 2: Mine the search traffic data for seasonality

* Step 3: Relate the search traffic to stock price patterns

* Step 4: Create a time series model with Prophet

The following subsections detail these steps.

In [ ]:
!pip install pystan~=2.19.1.1
!pip install prophet


In [ ]:
# Import the required libraries and dependencies
import pandas as pd
from prophet import Prophet
import datetime as dt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## Step 1: Find Unusual Patterns in Hourly Google Search Traffic

The data science manager asks if the Google search traffic for the company links to any financial events at the company. Or, does the search traffic data just present random noise? To answer this question, pick out any unusual patterns in the Google search data for the company, and connect them to the corporate financial events.

To do so, complete the following steps:

1. Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Visualize the results. Do any unusual patterns exist?

2. Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?

In [ ]:
# Store the data in a Pandas DataFrame
# Set the "Date" column as the Datetime Index.

df_mercado_trends = pd.read_csv(
    "https://static.bc-edx.com/ai/ail-v-1-0/m8/lms/datasets/google_hourly_search_trends.csv",
    index_col='Date',
    parse_dates=True,
    infer_datetime_format=True
).dropna()

# Review the first and last five rows of the DataFrame
display(df_mercado_trends.head())
display(df_mercado_trends.tail())

In [ ]:
print(df_mercado_trends.index.min(), df_mercado_trends.index.max())
print(df_mercado_trends.head())
print(df_mercado_trends.tail())



In [ ]:
# Review the data types of the DataFrame using the info function
df_mercado_trends.info()

In [ ]:
# Review the data types of the DataFrame using the info function
df_mercado_trends.info()

# Review the summary statistics of the DataFrame
df_mercado_trends.describe()

# Plot the data to visualize the trends
df_mercado_trends.plot()

In [ ]:
# Filter for May 2020
# Filter for May 2020 using .loc[]
may_2020_data = df_mercado_trends.loc['2020-05']


# Visualize May 2020 Search Trends
may_2020_data.plot(figsize=(10, 6), title='May 2020 Search Trends for MercadoLibre')
plt.ylabel('Search Trends')
plt.show()

# Calculate Total Search Traffic for May 2020
total_may_2020_traffic = may_2020_data['Search Trends'].sum()

# Calculate Monthly Median of Search Traffic
monthly_traffic_median = df_mercado_trends.resample('M').sum().median()[0]

# Compare May 2020 to the Monthly Median
print(f"Total search traffic for May 2020: {total_may_2020_traffic}")
print(f"Monthly median search traffic: {monthly_traffic_median}")
is_may_higher = "Yes" if total_may_2020_traffic > monthly_traffic_median else "No"
print(f"Is May 2020's search traffic higher than the median? {is_may_higher}")


In [ ]:
# Check the type of the DataFrame's index to confirm it's a DatetimeIndex
print(df_mercado_trends.index)

# If the index is correct, slice the DataFrame using the .loc accessor
df_may_2020 = df_mercado_trends.loc['2020-05']

# Visualize the sliced data
df_may_2020.plot(title='Google Search Trends for Mercado - May 2020', figsize=(12, 8))

In [ ]:
# Slice the DataFrame to just the month of May 2020
df_may_2020 = df_mercado_trends.loc['2020-05-01':'2020-05-31']

# Now let's try plotting this data again
df_may_2020.plot(title='Google Search Trends for Mercado - May 2020', figsize=(12, 8))

#### Step 2: Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?

In [ ]:
# Assuming df_mercado_trends is your DataFrame with the 'Search Trends' column

# Step 1: Calculate the total search traffic for May 2020
total_may_2020_traffic = df_mercado_trends.loc['2020-05', 'Search Trends'].sum()

# Step 2: Calculate the monthly median search traffic
# First, resample the data to monthly frequency, summing the search traffic
monthly_traffic = df_mercado_trends['Search Trends'].resample('M').sum()
# Then, calculate the median of the monthly totals
monthly_median_traffic = monthly_traffic.median()

# Step 3: Compare the May 2020 total to the monthly median
is_may_higher = total_may_2020_traffic > monthly_median_traffic

# Print the results
print(f"Total search traffic for May 2020: {total_may_2020_traffic}")
print(f"Monthly median search traffic: {monthly_median_traffic}")
print(f"Is May 2020's search traffic higher than the median? {'Yes' if is_may_higher else 'No'}")


In [ ]:
# Calculate the sum of the total search traffic for May 2020
traffic_may_2020 = df_may_2020['Search Trends'].sum()

# View the traffic_may_2020 value
print("Total search traffic for May 2020:", traffic_may_2020)

# Calculate the sum of the total search traffic for the entire dataset
total_traffic = df_mercado_trends['Search Trends'].sum()

# View the total_traffic value
print("Total search traffic for the entire dataset:", total_traffic)

# Calculate the percentage of the total search traffic for May 2020
percentage_may_2020 = (traffic_may_2020 / total_traffic) * 100

# View the percentage_may_2020 value
print("Percentage of total search traffic for May 2020:", percentage_may_2020)

# Reset the index of the DataFrame
df_mercado_trends = df_mercado_trends.reset_index()

# Review the first and last five rows of the DataFrame
display(df_mercado_trends.head())
display(df_mercado_trends.tail())

In [ ]:
# Divide the total traffic for May 2020 by the median monthly traffic
comparison_ratio = traffic_may_2020 / monthly_median_traffic

# View the comparison ratio
print("Comparison ratio of May 2020 traffic to overall monthly median traffic:", comparison_ratio)

## Step 2: Mine the Search Traffic Data for Seasonality

Marketing realizes that they can use the hourly search data, too. If they can track and predict interest in the company and its platform for any time of day, they can focus their marketing efforts around the times that have the most traffic. This will get a greater return on investment (ROI) from their marketing budget.

To that end, you want to mine the search traffic data for predictable seasonal patterns of interest in the company. To do so, complete the following steps:

1. Group the hourly search data to plot the average traffic by the hour of day. Does the search traffic peak at a particular time of day or is it relatively consistent?

2. Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday). Does the search traffic get busiest on any particular day of the week?

3. Group the hourly search data to plot the average traffic by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

In [ ]:
# Show Dataframe
df_mercado_trends.head()

In [ ]:
# Assuming 'df_mercado_trends' is your DataFrame and it contains a 'Date' column

# Convert 'Date' column to datetime type (if not already done)
df_mercado_trends['Date'] = pd.to_datetime(df_mercado_trends['Date'])

# Set 'Date' column as the DataFrame's index
df_mercado_trends.set_index('Date', inplace=True)

# Now, group by the hour of the index and calculate the average traffic
hourly_average_traffic = df_mercado_trends.groupby(df_mercado_trends.index.hour).mean()

# Plot the average traffic by the hour of the day
hourly_average_traffic['Search Trends'].plot(title='Average Hourly Search Traffic for Mercado', figsize=(12, 8))

# Show the plot
plt.show()


In [ ]:


# Sample data
data = {
    'day_of_week': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    'traffic': [100, 150, 200, 180, 220, 250, 210]  # Example average traffic values
}

# Create DataFrame
hourly_traffic = pd.DataFrame(data)

# Group the hourly search data by the day of the week and calculate the average traffic
average_weekly_traffic = hourly_traffic.groupby('day_of_week')['traffic'].mean()

# Set the x-tick labels
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(average_weekly_traffic, marker='o', linestyle='-')
plt.title('Average Weekly Traffic')
plt.xlabel('Day of the Week')
plt.ylabel('Average Traffic')
plt.xticks(range(7), days)  # Set x-tick labels
plt.grid(True)
plt.tight_layout()

# Display the plot
plt.show()




In [ ]:
# Group the hourly search data to plot the average traffic by the week of the year
# Group the DataFrame by the index week of the year and chain the mean function

average_weekly_traffic = df_mercado_trends.groupby(df_mercado_trends.index.isocalendar().week).mean()

# Now you can view or plot the average_weekly_traffic DataFrame as needed


# View the average_weekly_traffic DataFrame
average_weekly_traffic

# Plot the average weekly traffic
ax = average_weekly_traffic.plot(title='Average Hourly Search Traffic by Week of the Year', figsize=(12, 8))






**Question:** Are there any time based trends that you can see in the data?

**Answer:**

Based on the data visualizations provided:

1. **Daily Trends**: There is a clear daily pattern where search interest peaks during certain hours and dips during others, suggesting varying user engagement throughout the day.

2. **Weekly Trends**: The search traffic starts higher at the beginning of the week and gradually declines towards the weekend, indicating more user engagement on weekdays.

3. **Annual Trends**: Throughout the year, search traffic exhibits fluctuations with no clear trend during the winter holiday period, suggesting that external factors or events may influence search trends during different weeks of the year.

## Step 3: Relate the Search Traffic to Stock Price Patterns

You mention your work on the search traffic data during a meeting with people in the finance group at the company. They want to know if any relationship between the search data and the company stock price exists, and they ask if you can investigate.

To do so, complete the following steps:

1. Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

2. Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

3. Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

    * “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

    * “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis

4. Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

In [ ]:
# Upload the "mercado_stock_price.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the Datetime Index.
df_mercado_stock = pd.read_csv(
    "https://static.bc-edx.com/ai/ail-v-1-0/m8/lms/datasets/mercado_stock_price.csv",
    index_col="date",
    parse_dates=True
).dropna()


# View the first and last five rows of the DataFrame
display(df_mercado_stock.head())
display(df_mercado_stock.tail())




In [ ]:
# Visualize the closing price of the df_mercado_stock DataFrame
df_mercado_stock['close'].plot(title='Mercado Libre Stock Price', figsize=(12, 8))

# Check the type of the DataFrame's index to confirm it's a DatetimeIndex
print(df_mercado_stock.index)

In [ ]:
# Concatenate the df_mercado_stock DataFrame with the df_mercado_trends DataFrame
# Concatenate the DataFrame by columns (axis=1), and drop and rows with only one column of data
df_mercado = pd.concat([df_mercado_stock, df_mercado_trends], axis=1).dropna()


# View the first and last five rows of the DataFrame
display(df_mercado.head())
display(df_mercado.tail())

# Review the summary statistics of the DataFrame
df_mercado.describe()

# Visualize the closing price of the df_mercado DataFrame
df_mercado['close'].plot(title='Mercado Libre Stock Price', figsize=(12, 8))

In [ ]:
# Slice the data to just the first half of 2020
first_half_2020 = df_mercado_stock['2020-01':'2020-06']

# Plot the data
first_half_2020['close'].plot(title='Mercado Libre Stock Price - First Half of 2020', figsize=(12, 8))


In [ ]:
# Slice the DataFrame to the first half of 2020
df_first_half_2020 = df_mercado.loc['2020-01-01':'2020-06-30']

# Plot both the closing stock price and search trends
ax = df_first_half_2020['close'].plot(title='MercadoLibre Stock Price and Search Trends (First Half of 2020)', figsize=(14, 7), legend=True)
df_first_half_2020['Search Trends'].plot(secondary_y=True, ax=ax, legend=True)

# Label the primary and secondary y-axes
ax.set_ylabel('Closing Stock Price')
ax.right_ax.set_ylabel('Search Trends')

# Show the plot
plt.show()

In [ ]:
# Visualize the close and Search Trends data
# Plot each column on a separate axes using the following syntax
# `plot(subplots=True)`
df_mercado[['close', 'Search Trends']].plot(subplots=True, figsize=(12, 8))

# Display the plot
plt.show()

### Answer this Question: ###

## Do both time series indicate a common trend that’s consistent with this narrative? ##

Analysis suggests that there is some correlation between search trends and stock prices, particularly around significant company events such as earnings releases. The observation that search traffic increased prior to a rise in stock price around the earnings announcement could indeed be worth further investigation. However, caution is warranted in interpreting this as a predictable pattern without conducting a more detailed time-series analysis.

To determine if this correlation holds consistently and whether search activity could serve as a leading indicator for stock price movements, further research is needed. A more detailed time-series analysis or event study focusing on multiple earnings release dates and their impact on both search trends and stock prices would provide deeper insights. This analysis would help in understanding whether search volume increases are a reaction to anticipated news releases or if they indeed precede changes in stock prices.


#### Step 3: Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

* “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

* “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Lagged Search Trends
# This column should shift the Search Trends information by one hour

# Step 3: Create a new column "Lagged Search Trends" by shifting the "Search Trends" column by one hour
df_mercado['Lagged Search Trends'] = df_mercado['Search Trends'].shift(1)

# Step 3: Create a new column "Stock Volatility" using an exponentially weighted four-hour rolling average
# Calculate the stock volatility using the percent change in the closing price
# Then, apply an exponentially weighted moving average with a span of 4 hours
df_mercado['Stock Volatility'] = df_mercado['close'].pct_change().ewm(span=4).std()

# Step 3: Create a new column "Hourly Stock Return" by calculating the percent change of the stock price on an hourly basis
df_mercado['Hourly Stock Return'] = df_mercado['close'].pct_change()

# Display the updated DataFrame
display(df_mercado.head())
display(df_mercado.tail())

# Plot the stock volatility
df_mercado['Stock Volatility'].plot(title='Stock Volatility', figsize=(12, 8))

# Display the plot
plt.show()

In [ ]:
# Create a new column "Stock Volatility" using a 4-period rolling window standard deviation
df_mercado['Stock Volatility'] = df_mercado['close'].pct_change().rolling(window=4).std()

# Display the updated DataFrame
display(df_mercado.head())

# Plot the stock volatility
df_mercado['Stock Volatility'].plot(title='Stock Volatility', figsize=(12, 8))

# Display the plot
plt.show()

**Solution Note:** Note how volatility spiked, and tended to stay high, during the first half of 2020. This is a common characteristic of volatility in stock returns worldwide: high volatility days tend to be followed by yet more high volatility days. When it rains, it pours.

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Hourly Stock Return
# This column should calculate hourly return percentage of the closing price
# Create a new column "Hourly Stock Return" in the DataFrame
df_mercado['Hourly Stock Return'] = df_mercado['close'].pct_change() * 100

# Display the updated DataFrame
display(df_mercado.head())
display(df_mercado.tail())

#### Step 4: Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

In [ ]:
# Construct correlation table of Stock Volatility, Lagged Search Trends, and Hourly Stock Return
# Use the .corr() function to calculate the correlation between the columns
correlation = df_mercado[['Stock Volatility', 'Lagged Search Trends', 'Hourly Stock Return']].corr()

# View the correlation table
correlation

**Question:** Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

**Answer:** If we read down the second column (or read along the second row), we see that there's a slight negative correlation between searches for the firm and its subsequent stock volatility. More searches (slightly) tend to indicate less near-term hourly stock risk for the firm.

On the other hand, there's a positive correlation between search activity in one hour and stock returns in the next: as search activity goes up, so will the firm's stock price in the short term. This effect is very weak though, only half the size of the effect on stock volatility. With the correlation so close to zero, some might also say that there's almost no relationship between these two variables whatsoever, and that the small findings that we do have could be just due to chance. While there's no required threshold when looking at correlation between stock returns and the variables used to predict them, it does tend to be that correlations in this context tend to be very low across the board. It's hard to predict the market!

The correlation matrix you've provided summarizes the relationships between Stock Volatility, Lagged Search Trends, and Hourly Stock Return. Here's a breakdown of the correlations:

1. **Stock Volatility**:
   - Correlation with itself: 1.000000 (as expected)
   - Correlation with Lagged Search Trends: -0.148938
   - Correlation with Hourly Stock Return: 0.061424

2. **Lagged Search Trends**:
   - Correlation with Stock Volatility: -0.148938
   - Correlation with itself: 1.000000 (as expected)
   - Correlation with Hourly Stock Return: 0.017929

3. **Hourly Stock Return**:
   - Correlation with Stock Volatility: 0.061424
   - Correlation with Lagged Search Trends: 0.017929
   - Correlation with itself: 1.000000 (as expected)

These correlation values confirm the observations made earlier: there's a slight negative correlation between Lagged Search Trends and Stock Volatility, and a weak positive correlation between Lagged Search Trends and Hourly Stock Return. However, the correlations are relatively weak, suggesting that while there may be some relationship, it's not strong enough to make reliable predictions.

In [ ]:
# Review the data types of the DataFrame using the info function
df_mercado_trends.info()

# Review the summary statistics of the DataFrame
df_mercado_trends.describe()

# Plot the data to visualize the tre

In [ ]:
# Rename the index to "ds"
df_mercado_trends.index.name = 'ds'

# Reset index to convert datetime index to a column
df_mercado_trends.reset_index(inplace=True)

# Rename the "Search Trends" column to "y"
df_mercado_trends.rename(columns={'Search Trends': 'y'}, inplace=True)

# Initialize Prophet model
model = Prophet()

# Fit the model to the data
model.fit(df_mercado_trends)

# Make a forecast
future = model.make_future_dataframe(periods=24, freq='H')  # Forecasting for the next 24 hours
forecast = model.predict(future)

# Plot the forecast
fig = model.plot(forecast)






#### Step 2: After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

In [ ]:
# Make a forecast
future = model.make_future_dataframe(periods=24, freq='H')  # Forecasting for the next 24 hours
forecast = model.predict(future)

# Plot the forecast
fig = model.plot(forecast)


In [ ]:
# Create a future dataframe to hold predictions
# Make the prediction go out as far as 2000 hours (approx 80 days)
# Make a forecast
future = model.make_future_dataframe(periods=2000, freq='H')  # Forecasting for the next 2000 hours
forecast = model.predict(future)



# View the last five rows of the future_mercado_trends DataFrame
display(future.tail())

# View the last five rows of the forecast_mercado_trends DataFrame
display(forecast.tail())

# Plot the forecast
fig = model.plot(forecast)

# describe the forecast
forecast.describe()


**Question:**  How's the near-term forecast for the popularity of MercadoLibre?

**Answer:**
Based on the forecast generated by the Prophet model, the near-term popularity of MercadoLibre is expected to remain relatively stable. However, without specific numerical data from the forecast, it's challenging to provide exact details about the expected trend. For a more precise analysis, we would need to examine the forecasted values for the popularity of MercadoLibre over the next few hours or days.

#### Step 3: Plot the individual time series components of the model to answer the following questions:

* What time of day exhibits the greatest popularity?

* Which day of the week gets the most search traffic?

* What's the lowest point for search traffic in the calendar year?

In [ ]:
from prophet.plot import plot_components

# Plot the individual time series components
fig = model.plot_components(forecast)


1. **What time of day exhibits the greatest popularity?**
   - Looking at the bottom graph for daily seasonality, the time of day with the greatest popularity appears to be in the evening, just before midnight. There's a significant upswing leading to the peak at that time.

2. **Which day of the week gets the most search traffic?**
   - According to the second graph, which shows weekly seasonality, the day with the most search traffic seems to be Tuesday. It has the highest point on the weekly curve.

3. **What's the lowest point for search traffic in the calendar year?**
   - The third graph, showing yearly seasonality, indicates that the lowest point for search traffic is at the very beginning of the year, right after January 1st. It's the deepest trough on the annual curve.

These insights can be really telling for planning when to push advertisements, scheduling maintenance or updates for when traffic is lowest, and understanding customer behavior throughout the year.

In [ ]:
# Plot the forecast
plt.figure(figsize=(10, 6))
plt.plot(forecast.index[-2000:], forecast['yhat'][-2000:], label='yhat', color='blue')
plt.fill_between(forecast.index[-2000:], forecast['yhat_lower'][-2000:], forecast['yhat_upper'][-2000:], color='gray', alpha=0.3, label='yhat_lower to yhat_upper')
plt.xlabel('Date')
plt.ylabel('Values')
plt.title('Forecasted Values with Upper and Lower Bounds')
plt.legend()
plt.show()


In [ ]:
# Convert the 'ds' column to datetime type
forecast['ds'] = pd.to_datetime(forecast['ds'])

# Use the plot_components function to visualize the forecast results
fig = model.plot_components(forecast)





1. **Time of Day with Greatest Popularity:**
   - Looking at the bottom plot (Daily), the greatest popularity, or peak time, appears to be in the evening, right before the plot starts dropping. If we eyeball the x-axis, this peak seems to be around 8 PM to 9 PM.

2. **Day of Week with Most Search Traffic:**
   - In the second plot (Weekly), the highest point occurs at the beginning of the week. It looks like Monday is the day with the most search traffic, starting strong and then gradually decreasing over the week.

3. **Lowest Point for Search Traffic in the Calendar Year:**
   - The third plot (Yearly) shows the annual trends. The lowest point in the calendar year appears to be around the turn from October to November. This is when search traffic dips the most according to this plot.



Interesting article on Covid impact on Mecardo Libre: https://www.bloomberg.com/news/articles/2020-05-06/locked-down-latin-consumers-flock-online-and-boost-mercadolibre

In [ ]:
forecast.columns


In [ ]:
# Reset the index to make the datetime index a column
forecast.reset_index(inplace=True)

# Rename the datetime index column to 'ds'
forecast.rename(columns={'index': 'ds'}, inplace=True)

# Set the 'ds' column as the index
forecast.set_index('ds', inplace=True)

forecast.index





In [ ]:
# Reset the index to make the datetime index a column
forecast.reset_index(inplace=True)

# Set the 'ds' column as the index
forecast.set_index('ds', inplace=True)

forecast.index



In [ ]:
# Reset the index to make the datetime index a column
forecast.reset_index(inplace=True)

# Rename the datetime index column to 'ds'
forecast.rename(columns={'index': 'ds'}, inplace=True)

# Set the 'ds' column as the index
forecast.set_index('ds', inplace=True)

forecast.index



In [ ]:
print(forecast.columns)



In [ ]:
# Reset the index to make the datetime index a column
forecast.reset_index(inplace=True)

# Rename the datetime index column to 'ds'
forecast.rename(columns={'index': 'ds'}, inplace=True)

# Set the 'ds' column as the index
forecast.set_index('ds', inplace=True)

print(forecast.columns)

In [ ]:
# From the forecast_mercado_trends DataFrame, plot the data to visualize
#  the yhat, yhat_lower, and yhat_upper columns over the last 2000 hours
# Plot the forecast
fig = model.plot(forecast)